In [2]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import os

origin_df = pd.read_csv('/Users/ktg/Desktop/project/portfolio-streamlit/test/popup_name/data/message.csv')
origin_df.head()

,File,Message
0,/var/www/weven_supply/app_supply/web/content/m...,현재 비밀번호를 입력해주세요
1,/var/www/weven_supply/app_supply/web/content/m...,비밀번호를 다시한번 입력해주세요
2,/var/www/weven_supply/app_supply/web/content/m...,비밀번호를 다시한번 확인해주세요
3,/var/www/weven_supply/app_supply/web/content/m...,이메일 형식이 올바르지 않습니다.
4,/var/www/weven_supply/app_supply/web/content/m...,Email 형식이 아닙니다.


In [2]:
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7572 entries, 0 to 7571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   File     7572 non-null   object
 1   Message  7570 non-null   object
dtypes: object(2)
memory usage: 118.4+ KB


In [3]:
origin_df.describe()

,File,Message
count,7572,7570
unique,863,1717
top,/var/www/weven_supply/weven_service/asset/js/b...,적용할 대상이 선택되지 않았습니다.
freq,164,576


In [4]:
# 유니크한 메시지
unique_messages = origin_df['Message'].unique()
print(unique_messages[0:10])


['현재 비밀번호를 입력해주세요' '비밀번호를 다시한번 입력해주세요' '비밀번호를 다시한번 확인해주세요'
 '이메일 형식이 올바르지 않습니다.' 'Email 형식이 아닙니다.' '복구이메일주소 인증이 필요합니다.'
 '로그인이 필요한 페이지 입니다.' '아직 지원하지 않는 기능입니다.' '아이디(이메일)를 입력해 주세요.'
 '사용할 수 없는 아이디 입니다.(중복된 아이디입니다.)']


In [5]:
len(unique_messages)

1718

In [14]:
# 메시지 전처리한 칼럼 추가
origin_df['Message_preprocessed'] = origin_df['Message'].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
origin_df.head()

,File,Message,Message_preprocessed
0,/var/www/weven_supply/app_supply/web/content/m...,현재 비밀번호를 입력해주세요,현재비밀번호를입력해주세요
1,/var/www/weven_supply/app_supply/web/content/m...,비밀번호를 다시한번 입력해주세요,비밀번호를다시한번입력해주세요
2,/var/www/weven_supply/app_supply/web/content/m...,비밀번호를 다시한번 확인해주세요,비밀번호를다시한번확인해주세요
3,/var/www/weven_supply/app_supply/web/content/m...,이메일 형식이 올바르지 않습니다.,이메일형식이올바르지않습니다.
4,/var/www/weven_supply/app_supply/web/content/m...,Email 형식이 아닙니다.,Email형식이아닙니다.


In [26]:
!pip uninstall sentence_transformers

Found existing installation: sentence-transformers 3.2.1
Uninstalling sentence-transformers-3.2.1:
  Would remove:
    /Users/ktg/Desktop/project/portfolio-streamlit/.venv/lib/python3.11/site-packages/sentence_transformers-3.2.1.dist-info/*
    /Users/ktg/Desktop/project/portfolio-streamlit/.venv/lib/python3.11/site-packages/sentence_transformers/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [32]:
from openai import OpenAI
import asyncio
import pandas as pd
from pydantic import BaseModel
from dotenv import load_dotenv

load_dotenv()

True

In [39]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.47.1
    Uninstalling openai-1.47.1:
      Successfully uninstalled openai-1.47.1

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [76]:
# nest_asyncio 설정 (Jupyter 환경에서 필요)
import nest_asyncio
nest_asyncio.apply()

In [81]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import asyncio
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

import pandas as pd

# 출력 스키마 정의
response_schemas = [
    ResponseSchema(name="modified_text", description="Improved Korean phrasing"),
    ResponseSchema(name="score", description="Rating score for improved wording (0-9)")
]

# 출력 파서 설정
StrOutputParser = StructuredOutputParser.from_response_schemas(response_schemas)

# ChatGPT 모델 초기화
llm = ChatOpenAI(
    temperature=0.7,
    model="gpt-4o-2024-08-06"
)


In [107]:
result_example = f"""input: 현재 비밀번호를 입력해주세요.
output: 현재 사용 중인 비밀번호를 입력해 주세요.

input: 이메일형식이 올바르지 않습니다.
output: 이메일 형식이 올바르지 않습니다. 다시 확인해 주세요.

input: 한글이름을 입력해주세요.
output: 이름을 한글로 입력해 주세요.

input: 웹용 썸네일 이미지는 3MB를 넘을 수 없습니다.
output: 웹용 썸네일 이미지는 3MB 이하로 등록해 주세요.

input: 서비스 준비중입니다.
output: 현재 서비스 준비 중입니다. 잠시만 기다려 주세요."""

template = f"""You are tasked with improving a given Korean phrase to make it more user-friendly and natural. Your goal is to follow the instructions carefully and provide an improved version of the phrase along with a self-evaluation score.

To improve the phrase:
1. 존댓말 사용
2. 명확한 안내
3. 부드러운 어조
4. 필요한 경우 추가 설명

After improving the phrase, evaluate your own work by assigning a score from 0 to 9, where 0 is the lowest and 9 is the highest. Consider how well you've addressed each of the given criteria in your improvement.

Example:
{result_example}"""

In [108]:
template

"You are tasked with improving a given Korean phrase to make it more user-friendly and natural. Your goal is to follow the instructions carefully and provide an improved version of the phrase along with a self-evaluation score.\n\nTo improve the phrase:\n1. 존댓말 사용\n2. 명확한 안내\n3. 부드러운 어조\n4. 필요한 경우 추가 설명\n\nAfter improving the phrase, evaluate your own work by assigning a score from 0 to 9, where 0 is the lowest and 9 is the highest. Consider how well you've addressed each of the given criteria in your improvement.\n\nExample:\ninput: 현재 비밀번호를 입력해주세요.\noutput: 현재 사용 중인 비밀번호를 입력해 주세요.\n\ninput: 이메일형식이 올바르지 않습니다.\noutput: 이메일 형식이 올바르지 않습니다. 다시 확인해 주세요.\n\ninput: 한글이름을 입력해주세요.\noutput: 이름을 한글로 입력해 주세요.\n\ninput: 웹용 썸네일 이미지는 3MB를 넘을 수 없습니다.\noutput: 웹용 썸네일 이미지는 3MB 이하로 등록해 주세요.\n\ninput: 서비스 준비중입니다.\noutput: 현재 서비스 준비 중입니다. 잠시만 기다려 주세요."

In [85]:
prompt = PromptTemplate.from_template(template=template)
chain = prompt | llm | StrOutputParser()

TypeError: 'StructuredOutputParser' object is not callable

In [99]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    temperature=0, 
    model_name="gpt-4o-mini"
)

template = "아래 질문에 대한 답변을 해주세요. \n{query}"
prompt = PromptTemplate.from_template(template=template)
chain = prompt | llm | StrOutputParser()

In [109]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class ResponseModel(BaseModel):
    modified_text: str
    score: int

def improve_phrase(phrase):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": template},
            {"role": "user", "content": phrase}
        ],
        response_format=ResponseModel,
    )   

    return completion.choices[0].message.parsed

In [110]:
unique_messages[0]

'현재 비밀번호를 입력해주세요'

In [112]:

res = improve_phrase(unique_messages[0])
res


ResponseModel(modified_text='현재 사용 중인 비밀번호를 입력해 주세요.', score=8)

In [113]:
# origin_df[:100]에 modified_text, score 추가
# improve_phrase를 한번만 호출하여 결과 저장
improved_results = origin_df[:100]['Message_preprocessed'].apply(improve_phrase)

In [114]:
origin_df[:100]['modified_text'] = improved_results.apply(lambda x: x.modified_text)
origin_df[:100]['score'] = improved_results.apply(lambda x: x.score)
origin_df[:100].head()

/var/folders/0n/0g9n7qgs1039jj2120nlt1yr0000gn/T/ipykernel_34656/881120333.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  origin_df[:100]['modified_text'] = improved_results.apply(lambda x: x.modified_text)
/var/folders/0n/0g9n7qgs1039j

,File,Message,Message_preprocessed
0,/var/www/weven_supply/app_supply/web/content/m...,현재 비밀번호를 입력해주세요,현재비밀번호를입력해주세요
1,/var/www/weven_supply/app_supply/web/content/m...,비밀번호를 다시한번 입력해주세요,비밀번호를다시한번입력해주세요
2,/var/www/weven_supply/app_supply/web/content/m...,비밀번호를 다시한번 확인해주세요,비밀번호를다시한번확인해주세요
3,/var/www/weven_supply/app_supply/web/content/m...,이메일 형식이 올바르지 않습니다.,이메일형식이올바르지않습니다.
4,/var/www/weven_supply/app_supply/web/content/m...,Email 형식이 아닙니다.,Email형식이아닙니다.


In [115]:
improved_results

0       modified_text='현재 사용 중인 비밀번호를 입력해 주세요.' score=9
1          modified_text='비밀번호를 다시 한번 입력해 주세요.' score=8
2         modified_text='비밀번호를 다시 한 번 확인해 주세요.' score=8
3     modified_text='이메일 형식이 올바르지 않습니다. 다시 한번 확인해 주시...
4     modified_text='입력하신 이메일의 형식이 올바르지 않습니다. 올바른 이메...
                            ...                        
95    modified_text='체험형 사이트 생성이 완료되었습니다. 이용해 주셔서 감사...
96            modified_text='사이트 아이디를 입력해 주세요.' score=7
97    modified_text='로그인이 완료되었습니다. 서비스를 이용해 주세요.' sc...
98                modified_text='아이디를 입력해 주세요.' score=8
99               modified_text='비밀번호를 입력해 주세요.' score=7
Name: Message_preprocessed, Length: 100, dtype: object

In [120]:
df = pd.read_csv("/Users/ktg/Desktop/project/portfolio-streamlit/test/result_batch_1.csv")
df.to_excel(
    "/Users/ktg/Desktop/project/portfolio-streamlit/test/result_batch_1.xlsx",
    index=False
)


In [121]:
df = pd.read_csv("/Users/ktg/Desktop/project/portfolio-streamlit/test/result_batch_2.csv")
df.to_excel(
    "/Users/ktg/Desktop/project/portfolio-streamlit/test/result_batch_2.xlsx",
    index=False
)
